# Задача о плескании жидкости

By Jonas Tjemsland, Andreas Krogen, Håkon Ånes and Jon Andreas Støvneng.

Last edited: 2016-03-13

---

Этот блокнот основан на задании, данном в теме *TMA4320 Введение в науку до витенскапелиге берегнингер* в NTNU [1]. Задание было подготовлено Джоном Вегардом Веносом и Антоном Евграфовым. Коды основаны на ответах Герта Магне Кнутсена, Даниэля Халворсена и Йонаса Тьемсланда. 

На этом уроке мы будем изучать математическую модель гармонического движения свободной поверхности жидкости в двумерных контейнерах. Численный подход основан на линейной модели, которая сводит задачу к задаче о собственных значениях Стеклова (см. Приложения ниже). Проблема анализируется с использованием исходных точек и точек коллокации.

Анализируемая проблема часто известна как "проблема хлюпанья". Концепции, представленные в этом блокноте, имеют ряд важных применений, таких как транспортировка жидкостей в больших контейнерах на судах, и множество повседневных аналогий, таких как проливание из кофейной чашки.

In [ ]:
# Import libraries
import numpy as np
from scipy import linalg as la
import matplotlib
import matplotlib.pyplot as plt
from IPython.core.display import HTML
%matplotlib inline

# We are going to get some errors from casting imaginary numbers to real numbers,
# because of numerical rounding errors, and divisions by zero in la.eig(). We
# therefore disable warning messages.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set common figure parameters
newparams = {'axes.labelsize': 8, 'axes.linewidth': 1, 'savefig.dpi': 200,
             'lines.linewidth': 1, 'figure.figsize': (8, 3),
             'ytick.labelsize': 7, 'xtick.labelsize': 7,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 7, 'legend.frameon': True, 
             'legend.handlelength': 1.5, 'axes.titlesize': 7,}
plt.rcParams.update(newparams)

## 1 Вступление

Мы собираемся изучить математическую модель, описывающую гармоническое движение жидкости $\Omega$ в контейнере $\Gamma_\text{B}\subset \partial \Omega$ со свободной поверхностью $\Gamma_\text{F}\subset \partial \Omega$. Для простоты мы рассмотрим эту задачу в двух измерениях
$$
\begin{align}
    \Delta \psi(x,y) &= 0,\quad(x,y) \in \Omega,\\
    \boldsymbol{n}\cdot\nabla \psi(x,y) &= 0, \quad(x,y) \in \Gamma_\text B,\\
    \qquad\;\boldsymbol{n}\cdot\nabla \psi(x,y) &= \lambda\psi(x,y),\quad(x,y) \in \Gamma_\text F,\\
\end{align}
$$
где $\boldsymbol{n}=[n_x,n_y]$ - единичный вектор, указывающий наружу из области $\Omega$ (и $\Delta=\nabla^2$ - оператор Лапласа). Эта задача известна как смешанная задача Стеклова на собственные значения для оператора Лапласа. Физически это можно интерпретировать как контейнер, который бесконечно длинен в направлении $z$(в плоскости экрана). Установка показана на рисунке ниже. 

![Setup](images/container.png)

Для произвольного $\lambda$ задача имеет тривиальное решение $\psi=0$. Значения $\lambda$, которые дают по крайней мере одно нетривиальное решение, называются собственными значениями задачи, а соответствующие решения $\psi$ называются собственными функциями. Собственные значения $\lambda>0$ связаны с резонансными частотами жидкости в контейнере, а собственные функции $\psi$ на $\Gamma_\text{F}$ задают форму волн на свободной поверхности,

\begin{equation}
h(x,t)=\frac{\omega}{g}\sin(\omega t)\psi(x,H),
\end{equation}
где $g$ - ускорение силы тяжести, а $\omega=\sqrt{g\lambda}$ - частота волны. Это соотношение и задача собственных значений Стеклова обсуждаются в приложениях ниже. В этой тетради мы собираемся нормализовать волны таким образом, чтобы амплитуда стала 1.

## 2 Представление численных решений.
Давайте определим функцию $\Psi:\mathbb{R}\backslash\{\boldsymbol 0\}\to \mathbb R$ как
\begin{equation}
    \Psi(x,y) = \log \sqrt{x^2+y^2}.
\end{equation}
Для заданного числа исходных точек $(x_1^s,y_1^s),...,(x_N^s,y_N^s) \in \mathbb R^2 \backslash(\Omega\cup \partial \Omega)$ мы определяем функцию
\begin{equation}
    \psi(x,y) = \sum_{j=1}^N \alpha_j \Psi(x-x_j^s,y-y_j^s),
\end{equation}
где $\alpha_j$ - неизвестные коэффициенты, которые мы хотим рассчитать. Эта функция является решением первого уравнения смешанной задачи Стеклова на собственные значения (уравнение Пуассона) для произвольных констант $\alpha_1,...,\alpha_N$.

Поэтому наша цель состоит в том, чтобы найти $\alpha_1,...,\alpha_N$ таким образом, что два последних уравнения приблизительно выполнялись.

## 3 Численное решение задачи.

Мы собираемся разместить $N$ (приблизительно) равномерно распределенных точек на границе $\partial \Omega$ и потребовать, чтобы два последних уравнения задачи были верны в этих точках. Затем мы надеемся, что уравнения будут аппроксимированы на всей границе. Эти точки являются так называемыми точками коллокации. Пусть $N_\text B$ - количество точек коллокации $(x_1^c,y_1^c),...,(x_{N_\text B}^c,y_{N_\text B}^c)$ на $\Gamma_\text B$ и $N_\text F$ количество точек коллокации $(x_{N_{\text B}+1}^c,y_{N_{\text B}+1}^c),...,(x_{N_\text B+N_\text F}^c,y_{N_\text B+N_\text F}^c)$ на $\Gamma_\text F$. Для каждой точки коллокации вдоль границы $\Gamma_\text B$ и $\Gamma_\text F$ можно использовать второе и третье уравнение соответственно для получения уравнений с $\alpha_j$ в качестве неизвестных. Это приведет к $N=N_\text B+N_\text F$ уравнениям и неизвестным.

Чтобы упростить наши численные расчеты, мы представим эту систему уравнений в виде матрицы $A\boldsymbol \alpha = \lambda B \boldsymbol \alpha$, где $\boldsymbol \alpha = [\alpha_1,...,\alpha_N]^\text T$. Если мы вставим $\psi(x,y)$ в задачу на собственные значения Стеклова, мы заметим, что $A$ и $B$ являются матрицами $N\times N$ 
$$
A_{ij}=\frac{n_x(x_i^c-x_j^s)+n_y(y_i^c-y_j^s)}{(x_i^c-x_j^s)^2+(y_i^c-y_j^s)^2},
$$
$$
B_{ij}=\begin{cases}
\log\sqrt{(x_i^c-x_j^s)^2+(y_i^c-y_j^s)^2}, &i=N_{\text{B}+1},...,N,\\
0, &\text{otherwise.}
\end{cases}
$$

Нетривиальное решение $\boldsymbol \alpha \neq 0$ для системы уравнений $(A-\lambda B)\boldsymbol \alpha = 0$ существует только в том случае, если матрица $A-\lambda B$ сингулярна. В этом случае мы называем решение $(\lambda,\boldsymbol \alpha)$ совокупностью собственных значений/собственных векторов для обобщенной задачи на собственные значения, задаваемой матрицами $(A,B)$. Она легко решается с помощью функции *eig()* , определенной в пакете python *scipy.linalg*.

## 4 Представление геометрии и определение точек источника и коллокации
Мы собираемся представить границу в виде параметризации
$$
\begin{align}
    \Gamma_\text B & = \{\boldsymbol C_\text B(\xi):0\leq \xi \leq \Xi_\text B\},\\
    \Gamma_\text F & = \{\boldsymbol C_\text F(\xi):0\leq \xi \leq \Xi_\text F\}, 
\end{align}
$$
где $\boldsymbol C_\text B$ и $\boldsymbol C_\text F$ являются кусочно дифференцируемыми функциями. Мы собираемся выбрать исходные точки длиной $\delta>0$ из коллокаций, расположенных снаружи контейнера

### 4.1 Геометрия контейнера

Двумерная коробка длиной $L$ и высотой $H$ может быть параметризована с помощью
$$
    \boldsymbol C_\text B(\xi)=\begin{cases}
    \boldsymbol e_y(H-\xi),\qquad\qquad\qquad 0\leq \xi<H,\\
    \boldsymbol e_x(\xi-H), \qquad\qquad\qquad\! H\leq xi<H+L,\\
    L\boldsymbol e_x + \boldsymbol e_y (\xi -L-H), \qquad H+L\leq\xi\leq 2H+L=\Xi_\text B,
    \end{cases}
$$
$$
    \boldsymbol C_\text F(\xi) = \boldsymbol e_x(L-\xi)+H\boldsymbol e_y, \qquad\qquad 0\leq \xi \leq L = \Xi_\text F. 
$$
Теперь мы знаем достаточно, чтобы создать функцию, которая вычисляет исходные точки и точки коллокации геометрии коробки. Обратите внимание, что точки коллокации не должны располагаться слишком близко к углам, где $\Gamma_\text B$ и $\Gamma_\text F$ являются прерывистыми.

In [ ]:
def csPointsBox(N,L,H,delta):
    """ Calculates the source points and the collocation points of the box geometry.
    The points are chosen uniformly on each side, such that the total number of points
    are even.
    Parameters:
        N:     int. Number of source and collocation points.
        L:     float. Length of the box.
        H:     float. Height of the box.
        delta: float. Distance between the source and collocation points.
    Returns:
        xc:    float arr. 2xN array of the collocation points on the container.
        xs:    float arr. 2xN array of the corresponding source points.
    """
    # Checks if N is even. If not, add one.
    if (N % 2):
        N = N + 1
        
    # Number of source and collocation points on the different sides. N=2*NL+2*NH.
    NH = int(round(N*H/(2*L + 2*H))) # On the vertical lines.
    NL = int(N/2 - NH) # On the horizontal lines.

    # Spacing between the source and collocation points.
    spacingL = L/NL
    spacingH = H/NH
    
    # The unit normal vector of the container.
    n = np.zeros((2,N))
    n[0,0:NH] = -np.ones(NH)
    n[0,NH + NL:2*NH + NL] = np.ones(NH)
    n[1,NH:NH + NL] = -np.ones(NL)
    n[1,2*NH + NL:N] = np.ones(NL)
    
    # Calculates the collocation points, and saves them in a 2xN matrix.
    xc = np.zeros((2,N))
    xc[0,NH:NH + NL] = np.arange(spacingL/2.0,L,spacingL)
    xc[0,NH + NL:2*NH + NL] = L*np.ones(NH)
    xc[0,2*NH + NL:N] = np.arange(spacingL/2.0,L,spacingL)
    xc[1,0:NH] = np.arange(H-spacingH/2.0,0.0,-spacingH)
    xc[1,NH + NL:2*NH + NL] = np.arange(spacingH/2,H,spacingH)
    xc[1,2*NH + NL:N] = H*np.ones(NL)
    
    # Calculates the source points, and saves them in a 2xN matrix.
    xs = xc + delta*n
    
    return xc, xs

Следующий код визуализирует исходные точки и геометрию. Чтобы получить хорошие результаты для окончательного решения, возможно, придется выбрать разные $N$ и $\delta$.

In [ ]:
L = 3.0 # Length of the box.
H = 4.0 # Height of the box.
N = 40 # Number of collocation and source points.
delta = 0.4 # Distance between the source and collocation points.

# Calculates the collocation points and the source points.
xc, xs = csPointsBox(N,L,H,delta)

# Visualize the results.
# Plotting the container and the fluid surface.
plt.plot([0,0,L,L],[H,0,0,H],'k',label=r'$\Gamma_B$'); # The container.
plt.plot([0,L],[H,H],'b',label=r'$\Gamma_F$'); # The fluid surface.
# Plotting collocation and source points.
plt.plot(xc[0],xc[1],'r*',label='Collocation points')
plt.plot(xs[0],xs[1],'g*',label='Source points')
# Figure texts.
plt.title('Collocation points and source points for the box geometry.')
plt.xlabel('x'), plt.ylabel('y'), plt.axis('equal'), plt.legend();

### 4.1 Круговая геометрия
Наша круговая геометрия будет параметризована с помощью
$$
\begin{align}
    \boldsymbol C_\text B &=  \boldsymbol e_x[x_0+R\sin(\xi+\theta/2)]
                            +\boldsymbol e_y[y_0+R\cos(\xi+\theta/2)],
                            \quad 0\leq \xi\leq 2\pi - \theta,\\
    \boldsymbol C_\text F &=  \boldsymbol e_x[x_0+R\sin(\theta/2)-xi]
                            +\boldsymbol e_y[y_0+R\cos(\xi+\theta/2)],
                            \quad 0< \xi< 2R\sin(\theta/2).
\end{align}
$$
Точки коллокации и точки источника могут быть вычислены с помощью следующей функции

In [ ]:
def csPointsCircle(N,x0,y0,theta,delta):
    """ 
        N:      int. Number of source and collocation points.
        delta:  float. Distance between the source and collocation points.
    Returns:
        xc:     float arr. 2xN array of the collocation points on the container.
        xs:     float arr. 2xN array of the corresponding source points.
        n:      float arr. The unit normal vectors at the collocation points.
    """
    # Number of source and collocation points on the container (NB) and the surface (NF).
    circumference = R*(2*np.pi-theta+2*np.sin(theta/2))
    NF = round(N*R*2*np.sin(theta/2)/circumference)
    NB = N-NF
    
    # Angular spacing between the collocation points on the container.
    dtheta = (2*np.pi-theta)/NB
    # Spacing between the collocation points on the surface.
    dF = 2*R*np.sin(theta/2)/NF
        
    # Calculates the collocation points, and saves them in a 2xN matrix.
    xc = np.zeros((2,N))
    # NB
    xi1 = np.arange(dtheta/2,2*np.pi-theta,dtheta)
    xc[0,0:NB] = x0 + R*np.sin(xi1 + theta/2)
    xc[1,0:NB] = y0 + R*np.cos(xi1 + theta/2)
    # NF
    xi2 = np.arange(dF/2,2*R*np.sin(theta/2),dF)
    xc[0,NB:N] = x0 + R*np.sin(theta/2) - xi2
    xc[1,NB:N] = y0 + R*np.cos(theta/2)
    
    # The unit normal vector of the container.
    n = np.zeros((2,N))
    n[0,0:NB] = np.sin(xi1 + theta/2)
    n[1,0:NB] = np.cos(xi1 + theta/2)
    n[1,NB:N] = 1
    
    # Calculates the source points, and saves them in a 2xN matrix.
    xs = xc + delta*n
    
    return xc, xs, n

Мы визуализируем результат так же, как и для геометрии коробки.

In [ ]:
R = 2 # Radius.
theta = np.pi/2 # Opening angle
x0, y0 = 2, 2 # Center of the circle.
N = 40 # Number of collocation and source points.
delta = 0.4 # Distance between the source and collocation points.

# Calculates the collocation points and the source points.
xc, xs, n = csPointsCircle(N,x0,y0,theta,delta)

# Visualize the results.
# Plotting the container and the fluid surface.
xi = np.linspace(0,2*np.pi-theta,100,endpoint=True)
plt.plot(x0 + R*np.sin(xi + theta/2), y0 + R*np.cos(xi + theta/2),'k',label=r'$\Gamma_B$'); # The container.
plt.plot([x0 + R*np.sin(theta/2), x0 - R*np.sin(theta/2)], 
         [y0 + R*np.cos(theta/2), y0 + R*np.cos(theta/2)],'b',label=r'$\Gamma_F$'); # The fluid surface.
# Plotting collocation and source points.
plt.plot(xc[0],xc[1],'r*',label='Collocation points')
plt.plot(xs[0],xs[1],'g*',label='Source points')
# Figure texts.
plt.title('Collocation points and source points for the circular geometry.')
plt.xlabel('x'), plt.ylabel('y'), plt.axis('equal'), plt.legend();

### 4.3 Общая геометрия
Мы собираемся определить общую геометрию с помощью интерполяции полиномиальной кривой набора точек данных. Полиномиальная интерполяция объясняется в модуле Полиномиальная интерполяция в [numfys](numfys.net). Мы собираемся использовать интерполяцию Лагранжа. 

Начинаем с определения функции, которая выполняет интерполяцию.

In [ ]:
def curveInterpolation(x_data,y_data,N):
    """ Interpolates a curve through the points (x_data,y_data) and returns a
    curve C(xi), where 0<xi<1.
    Parameters:
        x_data: float arr. All the x-values of the data set.
        y_data: float arr. All the y-values of the data set.
                x_data and y_data has to be of the same length.
        N:      float arr. Number of parameter values.
    Returns:
        [Cx,Cy]:float arr. The interpolation curve.
    """
    n = min(len(x_data),len(y_data))
    t = np.linspace(0,1,N,endpoint=True) # Parameter.
    
    # Chebychev nodes.
    a = (np.cos(np.pi/(2*n))-1)/(2*np.cos(np.pi/(2*n)))
    b = (np.cos(np.pi/(2*n))+1)/(2*np.cos(np.pi/(2*n)))
    t_i = (b + a)/2 + (b - a)/2*np.cos((2*n-2*np.linspace(1,n,n)+1)*np.pi/(2*n))
    
    # Performing the interpolation.
    L = [1]*n
    for i in range(0,n):
        for j in range(0,n):
            if (j!=i):
                L[j]=L[j]*(t - t_i[i])/(t_i[j] - t_i[i])
    Cx, Cy = t*0, t*0
    for i in range(0,n):
        Cx = Cx + x_data[i]*L[i]
        Cy = Cy + y_data[i]*L[i]

    return [Cx,Cy]

Как и прежде, нам нужно найти единичные нормальные векторы, чтобы можно было вычислить исходные точки. Таким образом, нам нужно вычислить производную от интерполяционной кривой. Это можно реализовать, например, путем выведения формулы интерполяции Лагранжа (см. Модуль полиномиальной интерполяции), которая дает
$$
\boldsymbol C'_\text B (t) = \sum_{j=1}^n\boldsymbol P_j L'_j(t),
$$
$$
L_j'(t)=\sum_{i=1,i\neq j}^n\frac{1}{t_j-t_i}\prod_{k=1, k\neq i,j}^n\frac{t-t_k}{t_j-t_k}.
$$
Используя тот факт, что $\boldsymbol n\cdot \boldsymbol C'=0\wedge ||\boldsymbol n||= 1$, можно легко вычислить нормальные единичные векторы $\boldsymbol n(t)=[n_x(t),n_y(t)]$. Давайте создадим функцию, которая вычисляет единичные нормальные векторы и производную интерполяционного полинома.

In [ ]:
def diffCurveInterpolation(x_data,y_data,N):
    """ Calculates the derivative of a curve that tnterpolates the points 
    (x_data,y_data) dC(t) and the unit normal vectors n(t).
    Parameters:
        x_data:    float arr. All the x-values of the data set.
        y_data:    float arr. All the y-values of the data set.
                   x_data and y_data has to be of the same length.
        N:         float arr. Number of parameter values.
    Returns:
        [dCx,dCy]: float arr. The derivative of the interpolation curve.
        [nx,ny]:   float arr. The unit normal vectors.
    """
    n = min(len(x_data),len(y_data))
    t = np.linspace(0,1,N,endpoint=True) # Parameter.
    
    # Chebychev nodes.
    a = (np.cos(np.pi/(2*n))-1)/(2*np.cos(np.pi/(2*n)))
    b = (np.cos(np.pi/(2*n))+1)/(2*np.cos(np.pi/(2*n)))
    t_i = (b + a)/2 + (b - a)/2*np.cos((2*n-2*np.linspace(1,n,n)+1)*np.pi/(2*n))
    
    # Calculating the derivative of the interpolation.
    dL = [0]*n
    for i in range(0,n):
        for j in range(0,n):
            if (j!=i):
                multi = np.ones(len(t))
                for k in range(0,n):
                    if (j!=k and i!=k and j!=i):
                        multi = multi*(t - t_i[k])/(t_i[j] - t_i[k])
                dL[j] = dL[j] + 1/(t_i[j] - t_i[i])*multi
    dCx, dCy = t*0, t*0
    for i in range(0,n):
        dCy = dCy + y_data[i]*dL[i]
        dCx = dCx + x_data[i]*dL[i]
    
    # Calculating the unit normal vectors.
    nx = dCy/np.sqrt(dCx**2+dCy**2)
    ny = -dCx/np.sqrt(dCx**2+dCy**2)
    
    return [dCx,dCy], [nx,ny]

Нам также необходимо выполнить численное интегрирование для вычисления длины контейнера и равномерного распределения точек коллокации. Существует множество пакетов для python, которые включают такие функции (например, scipy.integrate.simps ()), но мы собираемся создать свой собственный. Конкретней, мы собираемся использовать метод Симпсона и составной метод Симпсонов (многошаговый). Ознакомьтесь с модулями численного интегрирования в [numfys](numfys.net) для получения более подробной информации о методе Симпсона.

In [ ]:
def compositeSimpsonsMethod(y,a,b):
    """ Uses the composite Simpsons method to calculate the definite integral of y(x). The 
    input y is the y-values at some x-values, which are assumed to be uniformly distributed 
    between (and included) a and b.
    Parameters:
        y:    Float arr. y-values of the function evaluated at uniformly distributed x-values.
        a:    Float. Start value for integration.
        b:    Float. End value for integration.
    Returns:
        I:    Float. Approximation of the definite integral.
    """
    n = len(y)-1 # Number of sub-intervals
    h = (b-a)/n # The assumed distance between the function evaluations y(x).
    # Performing the composite simpsons method.
    odd, even = 0, 0
    for i in range(1,n,2):
        odd += y[i]
    for i in range(2,n,2):
        even += y[i]
    I = h/3*(y[0] + y[n] + 4*odd + 2*even)
    return I

def simpleSimpsonsMethod(y0,y1,y2,dx):
    """ Uses the Simpsons method to calculate the definite integral of y(x) using the three
    function evaluations y0, y1, y2 at some x-values with a distance dx.
    Parameters:
        y0: Float. First y-value.
        y1: Float. Second y-value.
        y2: Float. Third y-value.
        dx: Float. Assumed destance between the function evaluations.
    Returns:
        Float. Approximation of the definite integral.
    """
    return dx/3*(y0+4*y1+y2)

Теперь у нас есть инструменты для определения контейнера из набора точек данных, равномерного распределения точек коллокации и последующего вычисления исходных точек. 

Для простоты предположим, что первая и последняя точки данных находятся на одном и том же горизонтальном уровне. Затем $\Gamma_\text F$ просто определяется как линия между этими двумя точками. Точки коллокации в $\Gamma_\text F$ и соответствующие исходные точки затем вычисляются, как и ранее. 

$N$ точек коллокации равномерно распределено между $\Gamma_\text F$ и $\Gamma_\text B$. Длина $\Gamma_\text B$ определяется путем интегрирования производной кривой,
$$
L_B = \int_{t_0}^{t_1} ||\boldsymbol C'_\text B(t)|| \,\text dt.
$$

Точки коллокации $\Gamma_\text B$ распределяются по следующему алгоритму:
Пусть $L_\text B$ - длина $\Gamma_\text B$. Затем значение параметра между каждой точкой коллокации задается значением $L_\text B/N_\text B$, которое, в свою очередь, может быть вычислено с помощью
$$
\int_{\hat{t}_{i-1}}^{\hat{t}_i} ||\boldsymbol C'_\text B(t)|| \,\text dt = \frac{L_\text B}{N_\text B}.
$$
Цель состоит в том, чтобы вычислить $\hat t_i$, значение параметра, при котором должна быть размещена точка коллокации $i$. Это может быть достигнуто, например, путем выполнения метода Симпсона между тремя значениями параметров, а затем проверки правильности интеграла. Первая точка коллокации должна быть расположена на расстоянии $L_\text B/2 N_\text B$ от начала.

Когда все это улажено, мы, наконец, готовы создать функцию, которая вычисляет точки коллокации и исходные точки общей геометрии.

In [ ]:
def csPointsGeneral(x_data,y_data,N,delta,Nt):
    """ Calculates source and collocation points of the general geometry. The general geometry
    is defined as the polynomial interpolation of (x_data,y_data). The first and the last element
    of y_data is assumed to be equal, such that a horizontal line between the first and the last
    data point can be achieved.
    Parameters:
        x_data: float arr. All the x-values of the data set.
        y_data: float arr. All the y-values of the data set.
                x_data and y_data has to be of the same length.
        N:      int. Number of collocation and source points.    
        delta:  float. Distance between the source and collocation points.
        Nt:     float. Number of parameter increments of the curve describing the general geometry.
    Returns:
        xc:     float arr. 2xN array of the collocation points on the container.
        xs:     float arr. 2xN array of the corresponding source points.
        [nx,ny]:float arr. The unit normal vectors at the collocation points.
        NB:     int. Number of collocation points on the container.
    
    """
    n = min(len(x_data),len(y_data))
    
    t = np.linspace(0,1,Nt,endpoint=True) # Parameter.
    dt = 1/Nt # Parameter increment.
    
    # The curve describing the container.
    [CBx,CBy] = curveInterpolation(x_data,y_data,Nt)
    # The derivative of the container and its unit normal vectors.
    [dCxB,dCyB], [nxB,nyB] = diffCurveInterpolation(x_data,y_data,Nt)
    
    # Length of the container.
    vel = np.sqrt(dCyB**2+dCxB**2)
    LB = compositeSimpsonsMethod(vel,0,1)
    # Length of the fluid surface.
    LF = abs(x_data[0]-x_data[n-1])
    
    # Distribution of collocation points.
    NB = int(N*LB/(LB+LF))
    NF = N-NB
    
    # Using Simpsons method to distibute the collocation points on the container as described
    # in the text.
    tHat = [0]*NB # The parameter indices at which the collocation points are to be placed.
    count = 0
    # First collocation point.
    I = 0
    while (I<=LB/(2*NB)):
        I += simpleSimpsonsMethod(vel[count],vel[count + 1],vel[count + 2],dt)
        count += 2
    tHat[0] = count - 1
    # The rest of the collocation points.
    for j in range(1,NB):
        I = 0
        while (I<=LB/(NB) and count < Nt-2):
            I += simpleSimpsonsMethod(vel[count],vel[count + 1],vel[count + 2],dt)
            count += 2
        tHat[j] = count - 1
        
    # The collocation points.
    xc = np.zeros((2,N))
    # On the container.
    xc[0,0:NB], xc[1,0:NB] = CBx[tHat], CBy[tHat]
    # On the fluid surface.
    xc[0,NB:N], xc[1,NB:N] = x_data[0] + np.arange(LF/(2*NF),LF,LF/NF), y_data[0]
    
    # The unit normal vectors.
    n = np.zeros((2,N))
    # On the container.
    n[0,0:NB], n[1,0:NB] = nxB[tHat], nyB[tHat]
    # On the fluid surface.
    n[0,NB:N], n[1,NB:N] = 0, 1
    
    # The source points.
    xs = n*delta + xc
    
    return xc, xs, n, NB

Наконец-то пришло время визуализировать исходные и коллокационные точки для общей геометрии, и не в последнюю очередь для самой геометрии.

In [ ]:
# Define some data points.
x_data = [1.5,0.8,0.8,1.2,1.5,2.5,3.6,3.8,4.2,4.1,3.8]
y_data = [4.5,3.7,2.5,1.5,0.9,0.4,1.0,2.0,3.1,4.0,4.5]
n = len(x_data) # Number of data points.
N = 40 # Number of collocation and source points.
delta = 0.4 # Distance between the source and collocation points.
Nt = N**3 # Number of parameter increments of the curve describing the general geometry.

# Calculating the source and collocation points. 
xc, xs, dummy, dummy = csPointsGeneral(x_data,y_data,N,delta,Nt)

# Visualize the results.
# Plotting the container and the fluid surface.
[Cx,Cy] = curveInterpolation(x_data,y_data,100)
plt.plot(Cx,Cy,'k',label=r'$\Gamma_B$')
plt.plot([x_data[0],x_data[n-1]],[y_data[0],y_data[n-1]],'b',label=r'$\Gamma_B$')
# Plotting collocation and source points.
plt.plot(xc[0],xc[1],'r*',label='Collocation points')
plt.plot(xs[0],xs[1],'g*',label='Source points')
# Figure texts.
plt.title('Collocation points and source points for a general geometry.')
plt.xlabel('x'), plt.ylabel('y'), plt.axis('equal'), plt.legend();

# Plot the data points and set figure texts.
plt.plot(x_data,y_data,'m.',label='Data points.')
plt.title('Polynomial curve interpolation of a set data points.')
plt.legend(), plt.xlabel('x'), plt.ylabel('y');

## 5 Решение смешанной задачи на собственные значения Стеклова и построение графиков волн

Теперь все готово для решения смешанной задачи о собственных значениях Стеклова для нашей установки. Как упоминалось ранее, мы собираемся использовать функцию *scipy.linalg.eig()* для решения задачи на собственные значения $A\boldsymbol \alpha = \lambda B \boldsymbol \alpha$ определенной ранее. Заметьте, что $B$ является единственной (что подразумевает, что $B\boldsymbol{\alpha} = 0$ имеет нетривиальные решения), но $A-\lambda B \approx -\lambda B$ для больших $|\lambda|$. Как следствие, scipy.linalg.eig() может генерировать собственные значения $\pm \infty$. Мы собираемся отбросить эти собственные значения и отсортировать остальные (конечные) значения таким образом, чтобы мы могли легко анализировать волны, соответствующие наименьшим собственным значениям.

### 5.1 Геометрия коробки. Аналитическое решение
Для начала давайте найдем точки коллокации, исходные точки и матрицы $A$ и $B$, как описано ранее. $N$ и $\delta$ выбираются после некоторых проб и ошибок.

In [ ]:
L = 3.0 # Length of the box.
H = 4.0 # Height of the box.
N = 100 # Number of collocation and source points.
delta = 0.5 # Distance between the source and collocation points.

# Calculates the collocation points and the source points.
xc, xs = csPointsBox(N,L,H,delta)

# Number of source and collocation points on the different sides. N=2*NL+2*NH.
NH = int(round(N*H/(2*L + 2*H))) # On the vertical lines.
NL = int(N/2 - NH) # On the horizontal lines.

# The unit normal vector of the container in the collocation points.
n = np.zeros((2,N))
n[0,0:NH] = -np.ones(NH)
n[0,NH + NL:2*NH + NL] = np.ones(NH)
n[1,NH:NH + NL] = -np.ones(NL)
n[1,2*NH + NL:N] = np.ones(NL)

# The A and B matrices.
A = np.zeros((N,N))
B = np.zeros((N,N))
for i in range(0,N):
    A[i,:] = ((n[0,i]*(xc[0,i] - xs[0,:]) + n[1,i]*(xc[1,i]-xs[1,:]))/
              ((xc[0,i] - xs[0,:])**2 + (xc[1,i] - xs[1,:])**2))
for i in range(2*NH + NL,2*NH + 2*NL):
    B[i,:] = np.log(np.sqrt((xc[0,i] - xs[0,:])**2 + (xc[1,i] - xs[1,:])**2))

scipy.linalg.eig() решает обычную или обобщенную задачу на собственные значения квадратной матрицы. В нашем случае мы хотим решить $A\boldsymbol \alpha = \lambda B \boldsymbol \alpha$

In [ ]:
# Solve the eigenvalue problem.
lamb,alph = la.eig(A,B,left=0,right=1)

Теперь мы хотим игнорировать $\lambda\leq0$ и $\lambda = \pm \infty$, а затем отсортировать собственные значения и соответствующие собственные векторы. Мы создаем функцию, которая делает это, чтобы мы не повторяться

In [ ]:
def validLambdaAlpha(lamb,alph):
    """ Seperates out unvalid eigenvalues and returns a sorted list of valid eigenvalues.
    Parameters:
        lamb:       float arr. List of eigenvalues.
        alph:       float arr. Array of eigenvectors corresponding to the eigevalues. 
                    Each row holds a eigevector.
    Returns:
        sortLamb:   float arr. sorted list of valid eigenvalues.
        sortAlph:   float arr. Array of sorted eigenvectors corresponding  to the eigevalues. 
                    Each row holds a eigevector.
    """
    # Check for valid eigenvalues, and save them and the corresponding eigenvectors in two arrays.
    # All of our eigenvalues should be real, but the eig-function returns
    # returns imaginary numbers (a+0j) for the eigenvalues.
    validAlph = [] # Valid eigenvectors and eigenvalues.
    validLamb = []
    count = 0 # Number of valid eigenvectors and eigenvalues.
    for i in range(0,N):
        if lamb[i] > 0 and np.isfinite(lamb[i]):
            validLamb.append(float(lamb[i]))
            validAlph.append(alph[:,i])
            count+=1

    # Sort the eigenvalues and eigenvector on the value of the eigenvalues.
    sortIndex = np.argsort(validLamb)
    sortLamb = validLamb.copy()
    sortAlph = validAlph.copy()
    for i in range(0,count):
        sortLamb[i] = validLamb[sortIndex[i]]
        sortAlph[i] = validAlph[sortIndex[i]]
    return sortLamb, sortAlph

Для этой геометрии коробки известно аналитическое решение. Это
$$
\psi_n(x,y)=A_n\cos\frac{n\pi x}{L}\cosh\frac{n\pi y}{L},
$$
с соответствующими собственными значениями
$$
\lambda_n = \frac{n\pi}{L}\tanh\frac{n\pi H}{L}.
$$
Это легко показать, используя, например, разделение переменных. Обратите внимание, что $A_n$ - это неизвестные параметры (амплитуды), но мы собираемся нормализовать волны таким образом, чтобы они получили амплитуду 1. Поскольку мы знаем аналитическое решение, мы собираемся сравнить численное решение и аналитическое решение.

In [ ]:
# The sorted numerical eigenvalues and eigenvectors.
validLamb, validAlph = validLambdaAlpha(lamb,alph)
# The analytical eigenvalues.
analyticalLamb = np.arange(1,len(validLamb) + 1)*np.pi/L*np.tanh(
                                np.arange(1,len(validLamb) + 1)*np.pi*H/L)
# The relative error between the analytical and the numerical eigenvalues.
relError = abs((analyticalLamb - validLamb)/analyticalLamb)
# Printing the results as a HTML table.
table = ["<table>",
         "<td><b>Node</b></td>",
         "<td><b>Numerical eigenvalue</b></td>",
         "<td><b>Analytitcal eigenvalue</b></td>",
         "<td><b>Relative error</b></td>"]
for i in range(0,min(len(validLamb),10)):
    table.append("<tr><td>")
    table.append(str(i + 1))
    table.append("</td><td>")
    table.append("%.10f" % float(validLamb[i]))
    table.append("</td><td>")
    table.append("%.10f" % float(analyticalLamb[i]))
    table.append("</td><td>")
    table.append("%.5f %%" % (float(relError[i])*100))
    table.append("</td></tr>")
table.append("</table>")
HTML(''.join(table))

Давайте построим график волн!

In [ ]:
x = np.linspace(0,L,100,endpoint=True) # x-axis.
psi = np.zeros(len(x)) # Allocate the wave function, psi.

for node in [1,2,3,4]:
    # Calculate the wave fucntion using.
    psi = np.zeros(len(x))
    for n in range(0,N):
        psi = psi + validAlph[node - 1][n]*np.log(np.sqrt(
            (x - xs[0,n])**2 + (H - xs[1,n])**2 ));
    # Calculate h and normalize such that the amplitude becomes 1.
    h = psi/np.max(psi)*np.sign(psi[0])

    # Calculate the theoretical curve.
    psiT = np.cos(node*np.pi*x/L)*np.cosh(node*np.pi*H/L)
    hT = psiT/np.max(psiT)*np.sign(psiT[0])

    # Calculate the maximal absolute error of the normalized h functions.
    absError = np.max(np.abs(h-hT))

    # Plot the result.
    plt.figure()
    plt.plot(x,hT+H,'r',label='Analytical solution.')
    plt.plot(x,h+H,'b',label='Numerical solution.')
    plt.xlabel('x'); plt.ylabel('y');
    plt.title('%d. node.' % node) ,plt.legend();

### 5.2 Круговая геометрия.
Теперь мы повторяем процесс для циркулярной геометрии.

In [ ]:
R = 2 # Radius.
theta = np.pi/2 # Opening angle
x0, y0 = 2, 2 # Center of the circle.
N = 100 # Number of collocation and source points.
delta = 0.5 # Distance between the source and collocation points.
H = y0 + R*np.cos(theta/2)
# Number of source and collocation points on the container (NB) and the surface (NF).
circumference = R*(2*np.pi-theta+2*np.sin(theta/2))
NF = int(round(N*R*2*np.sin(theta/2)/circumference))
NB = N-NF

# Calculates the collocation points and the source points.
xc, xs, n = csPointsCircle(N,x0,y0,theta,delta)

# The A and B matrices.
A = np.zeros((N,N))
B = np.zeros((N,N))
for i in range(0,N):
    A[i,:] = ((n[0,i]*(xc[0,i] - xs[0,:]) + n[1,i]*(xc[1,i]-xs[1,:]))/
              ((xc[0,i] - xs[0,:])**2 + (xc[1,i] - xs[1,:])**2))
for i in range(NB,N):
    B[i,:] = np.log(np.sqrt((xc[0,i] - xs[0,:])**2 + (xc[1,i] - xs[1,:])**2))
    
# Solve the eigenvalue problem.
lamb,alph = la.eig(A,B,left=0,right=1)

# The sorted numerical eigenvalues and eigenvectors.
validLamb, validAlph = validLambdaAlpha(lamb,alph)
# The analytical eigenvalues.
analyticalLamb = np.arange(1,len(validLamb) + 1)*np.pi/L*np.tanh(
                                np.arange(1,len(validLamb) + 1)*np.pi*H/L)
# The relative error between the analytical and the numerical eigenvalues.
relError = abs((analyticalLamb - validLamb)/analyticalLamb)
# Printing the results as a HTML table.
table = ["<table>",
         "<td><b>Node</b></td>",
         "<td><b>Numerical eigenvalue</b></td>"]
for i in range(0,min(len(validLamb),10)):
    table.append("<tr><td>")
    table.append(str(i + 1))
    table.append("</td><td>")
    table.append("%.10f" % float(validLamb[i]))
    table.append("</td></tr>")
table.append("</table>")
HTML(''.join(table))

In [ ]:
x = np.linspace(x0 - R*np.sin(theta/2), x0 + R*np.sin(theta/2),100,endpoint=True) # x-axis.
psi = np.zeros(len(x)) # Allocate the wave function, psi.

for node in [1,2,3,4]:
    # Calculate the wave fucntion using.
    psi = np.zeros(len(x))
    for n in range(0,N):
        psi = psi + validAlph[node - 1][n]*np.log(np.sqrt(
            (x - xs[0,n])**2 + (H - xs[1,n])**2 ));
    # Calculate h and normalize such that the amplitude becomes 1.
    h = psi/np.max(psi)*np.sign(psi[0])

    # Plot the result.
    plt.figure()
    plt.plot(x,h+H,'b',label='Numerical solution.')
    plt.xlabel('x'); plt.ylabel('y');
    plt.title('%d. node.' % node), plt.legend();

### 5.3 Общая геометрия.
Теперь мы повторяем процесс для общей геометрии.

In [ ]:
# Define some data points.
x_data = [1.5,0.8,0.8,1.2,1.5,2.5,3.6,3.8,4.2,4.1,3.8]
y_data = [4.5,3.7,2.5,1.5,0.9,0.4,1.0,2.0,3.1,4.0,4.5]
n = len(x_data) # Number of data points.
N = 100 # Number of collocation and source points.
delta = 0.4 # Distance between the source and collocation points.
Nt = N**2 # Number of parameter increments of the curve.
H = y_data[0]

# Calculating the source and collocation points. 
xc, xs, n, NB = csPointsGeneral(x_data,y_data,N,delta,Nt)

# The A and B matrices.
A = np.zeros((N,N))
B = np.zeros((N,N))
for i in range(0,N):
    A[i,:] = ((n[0,i]*(xc[0,i] - xs[0,:]) + n[1,i]*(xc[1,i]-xs[1,:]))/
              ((xc[0,i] - xs[0,:])**2 + (xc[1,i] - xs[1,:])**2))
for i in range(NB,N):
    B[i,:] = np.log(np.sqrt((xc[0,i] - xs[0,:])**2 + (xc[1,i] - xs[1,:])**2))
    
# Solve the eigenvalue problem.
lamb,alph = la.eig(A,B,left=0,right=1)

# The sorted numerical eigenvalues and eigenvectors.
validLamb, validAlph = validLambdaAlpha(lamb,alph)
# The analytical eigenvalues.
analyticalLamb = np.arange(1,len(validLamb) + 1)*np.pi/L*np.tanh(
                                np.arange(1,len(validLamb) + 1)*np.pi*H/L)
# The relative error between the analytical and the numerical eigenvalues.
relError = abs((analyticalLamb - validLamb)/analyticalLamb)
# Printing the results as a HTML table.
table = ["<table>",
         "<td><b>Node</b></td>",
         "<td><b>Numerical eigenvalue</b></td>"]
for i in range(0,min(len(validLamb),10)):
    table.append("<tr><td>")
    table.append(str(i + 1))
    table.append("</td><td>")
    table.append("%.10f" % float(validLamb[i]))
    table.append("</td></tr>")
table.append("</table>")
HTML(''.join(table))

In [ ]:
x = np.linspace(x_data[0],x_data[len(x_data)-1],100,endpoint=True) # x-axis.
psi = np.zeros(len(x)) # Allocate the wave function, psi.

for node in [1,2,3,4]:
    # Calculate the wave fucntion using.
    psi = np.zeros(len(x))
    for n in range(0,N):
        psi = psi + validAlph[node - 1][n]*np.log(np.sqrt(
            (x - xs[0,n])**2 + (H - xs[1,n])**2 ));
    # Calculate h and normalize such that the amplitude becomes 1.
    h = psi/np.max(psi)*np.sign(psi[0])

    # Plot the result.
    plt.figure()
    plt.plot(x,h+H,'b',label='Numerical solution.')
    plt.xlabel('x'); plt.ylabel('y');
    plt.title('%d. node.' % node), plt.legend();

#### 6 Несколько заключительных замечаний

Увеличение количества точек источника и коллокаций приводит к повышению точности (попробуйте сами!). Это происходит за счет увеличения времени вычислений, поскольку это приводит (среди прочего) к увеличению числа уравнений в $A\boldsymbol \alpha = \lambda B \boldsymbol \alpha$. Это также дает увеличенное число допустимых собственных значений; большее количество собственных значений вычисляется с лучшим приближением.

Увеличение значения $\delta$ приводит к повышению точности для собственных значений с низким значением, но может привести к увеличению погрешности собственных значений с более высоким значением. Например, относительная погрешность наименьших собственных значений геометрии коробки с $N=200$ и $\delta = 1$ равна $1.000\%$, $0.000\%$, $0.000\%$ и $21.394\%$. 

$N=100$ и $\delta = 0.5$ выбраны в качестве компромисса вышеуказанных рассуждений и в то же время дают приемлемую ошибку. Поскольку круговая геометрия и общая геометрия находятся в том же порядке величины, что и прямоугольная геометрия, предполагается, что те же самые $N$ и $\delta$ дают хорошие приближения и для этих геометрий. Можно также проверить наши расчеты, тщательно увеличив $N$ (или $\delta$), и проверить, сходятся ли численные значения.

Потенциальное решение о том, какой контейнер является лучшим, должно основываться на обсуждениях высоких точек, амплитуд и собственных частот/резонансных частот. Кроме того, следует также учитывать силы, действующие на жидкость и на контейнер (особенно на большие контейнеры). Наименьшие собственные значения представляют наибольший физический интерес, поскольку приближения, описанные в приложениях, здесь наиболее точны.

Давайте определим оси, для которых единица измерения - м (метр). Тогда собственное значение имеет единицу m$^{-1}$, и мы можем использовать $g=9.81$ m/s$^2$. Из приложений мы видим, что $\lambda = \omega^2/g$. Например, первый узел геометрии коробки имеет в наших расчетах собственное значение $\lambda_1 = 1.016$ m$^{-1}$. Это соответствует резонансной частоте $f = \omega/2\pi = \sqrt{\lambda g}/2\pi \approx 0.502 $ s$^{-1}$. Давайте приведем более тривиальный пример. Предположим, что обычная кофейная чашка имеет цилиндрическую форму диаметром 6 см и высотой 10 см. Наши численные расчеты выполняются в двух измерениях, но было бы разумно предположить, что собственные частоты этого цилиндра будут иметь собственные значения того же порядка, что и двумерная геометрия коробки шириной 6 см и высотой 10 см. Это поле имеет собственное значение $\lambda_1 = 44.869$, что соответствует резонансной частоте $f=3.339$ s$^{-1}$. Будьте осторожны с тем, как быстро вы ходите со своей кофейной кружкой!

# A Вывод уравнений.
## A.1 Уравнения в частных производных для жидкости.
Предположим, что жидкость идеальна. Это означает, что она легко сжимается, имеет нулевую вязкость и имеет постоянное и равномерное распределение плотности. Уравнения Эйлера для жидкости тогда
$$
\sum_j \partial_j u_j = 0, \quad [\text{inkompressibility}]
$$
$$
\partial_t u_i + \sum_j u_j \partial_j u_i + \frac{1}{\rho}\partial_i p = a_i, \quad [\text{Newtons second law}]
$$
где $\boldsymbol u = [u_1,u_2,u_3]: \Omega \times \mathbb R\to \mathbb R$ скорость, $p:\Omega\times \mathbb R\to\mathbb R$ это давление, и $\boldsymbol a = [a_1,a_2,a_3]:\Omega \times\mathbb R\to\mathbb R$ ускорение (например, гравитационное ускорение). Предположим также, что $\partial_i u_j - \partial_ju_i = 0$ для всех $i,j = 1,2,3$ (предположение о потенциальном потоке). Затем скорость может быть записана как $u_i=\partial_i\psi$ для функции $\phi:\omega \times \mathbb R \to \mathbb R$ (разложение Ходжа), тогда мы получим
$$
\sum_j \partial_j ^2\phi = 0, \quad[\text{Уравнение Пуассона}]
$$
$$
\partial_i\partial_t\psi + \sum_j\partial_j\psi\partial_j\partial_i\psi + \frac{1}{\rho}\partial_ip = a_i.
$$
Если мы предположим, что $a_1=a_2=0$ и $a_3=-g$ (ускорение силы тяжести), то последнее уравнение можно интегрировать, чтобы получить уравнение Бернулли, которое определяет $p$ с учетом $\psi$
$$
\partial_t\psi + \frac{1}{2}\sum_j [\partial_j\psi]^2+\frac{p-p_\text{ref}}{\rho}+gx_3 = 0,
$$
где $p_\text{ref}$ - постоянное опорное давление (константа интегрирования).
## A.2 Граничные условия.
На границе мы знаем, что $\sum_jn_ju_j=V,$ где $V$ - перпендикулярная скорость на границе. Другими словами, в фиксированном контейнере $\Gamma_\text B$
$$
\sum_j n_j\partial_j \phi = 0,\quad (x_1,x_2,x_3)\in \Gamma_\text B.
$$
Свободная поверхность - $(x_1,x_2,H+h(x_1,x_2,t))$ для неизвестной функции $h(x_1,x_2,t)$. Мы знаем, что давление на свободной поверхности равно атмосферному давлению $p_\text{atm}$, и при $x_3=0$ давление равно $p=p_\text{atm} + \rho g H$. Поэтому мы вставляем $p_\text{ref}=p_\text{atm}+\rho g H$ в последнее уравнение A. 1 и получаем
$$
\partial_t \phi + \frac{1}{2}\sum_j [\partial_j\phi]^2+gh = 0,
$$
когда $x_3 = H+h(x_1,x_2,t)$. Обратите внимание, что амплитуда является функцией $x_1$, $x_2$ и $t$: $x_3 = x_3(x_1,x_2,t)$. Если мы выведем $x_3 = H+h(x_1,x_2,t)$ относительно $t$ и используем правило цепочки, мы получим
$$
u_3 = \frac{\text dx_3}{\text dt} = \frac{\text dh}{\text dt} = \partial_th+u_1\partial_1h+u_2\partial_2h.
$$
If we use the fact that $\boldsymbol u = \nabla \phi$, we get
$$
\partial_3 \phi = \partial_t h + \partial_1\phi\partial_1 h+\partial_2\phi\partial_2 h.
$$

## A.3 Линейные волны.
Мы предполагаем, что скорость $\boldsymbol u$, высота волны $h$ и колебания высоты волны $\nabla h$ невелики. То есть мы можем пренебречь нелинейными слагаемыми $[\partial_j\phi]^2$ и $\partial_i\phi\partial_ih$, и записать $\Gamma_\text F = \{(x_1,x_2,x_3)\,|\,x_3 = H\}$ вместо $\Gamma_\text F = \{(x_1,x_2,x_3)\,|\,x_3 = H+h(x_1,x_2,t)\}$. Другими словами, на свободной поверхности $\Gamma_\text F$ мы получаем
$$
\begin{align}
\partial_t\phi+gh &= 0,\\
\partial_t h -\partial_3 \phi &= 0,
\end{align}
$$
что подразумевает, что
$$
\partial_t^2\phi + g\partial_3\phi = \partial_t^2\phi + g\sum_j n_j\partial_j \phi = 0,
$$
так как $n=(0,0,1)$ на свободной поверхности $\Gamma_\text F = \{(x_1,x_2,H)\}$.

Теперь мы предполагаем,что $\phi(x_1,x_2,x_3,t) = \psi(x_1,x_2, x_3)\cos(\omega t)$, где частота $\omega$ постоянна. То есть мы предполагаем гармоническое движение. Это означает, что $\partial_t^2\phi=-\omega^2\phi$, и все уравнения могут быть переписаны как независимые от времени,
$$
\begin{align}
\sum_j \partial_j^2\psi &= 0, & (x_1,x_2,x_3)\in \Omega,\\
\sum_j n_j\partial_j\psi&=0, & (x_1,x_2,x_3)\in\Gamma_\text B,\\\
\sum_jn_j\partial_j\psi &= \omega^2g^{-1}\psi, \quad & (x_1,x_2,x_3)\in\Gamma_\text F.
\end{align}
$$
Когда мы знаем $\psi$ (и, следовательно, также $\phi$), $h$ может быть вычислен из первого уравнения A.3 как
$$
h(x_1,x_2,t)=g^{-1}\omega \sin(\omega t)\psi(x_1,x_2,H).
$$
Помните, что из-за сжимаемости третье уравнение A.3 выполняется только в том случае, если
$$
0 = \int_{\partial\Omega}\sum_jn_ju_j\;\text d\Gamma = \int_{\partial\Omega}\sum_jn_j\partial_j\psi\;\text d\Gamma = \int_{\Gamma_\text F}\sum_jn_j\partial_j\psi\;\text d\Gamma = \omega^2g^{-1}\int_{\Gamma_\text F}\psi\;\text d\Gamma.
$$
То есть, когда $\omega\neq 0$ будет $\int_{\Gamma_\text F}\psi \;\text d\Gamma = 0$. Это хороший способ проверить правильность наших численных расчетов.

## Ресурсы и дальнейшее чтение 
This Notebook is based on an assignment given in the subject *TMA4320 Introduksjon til vitenskapelige beregninger* at NTNU. The assignment was prepared by Jon Vegard Venås and Anton Evgrafov. The codes are based on the answers by Gjert Magne Knutsen, Daniel Halvorsen and Jonas Tjemsland. 

[1] Project description, https://wiki.math.ntnu.no/_media/tma4320/2016v/project_sloshing.pdf, (Last downloaded 10.04.2016)

scipy.linalg.eig: http://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eig.html, (aquired 05.03.16)